In [42]:
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score, roc_curve,auc
from sklearn.tree import DecisionTreeClassifier, plot_tree
#import gridsearchcv and lasso and kfold
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')
# Use GBC to predict the outcome
from sklearn.ensemble import GradientBoostingClassifier

In [78]:
training = pd.read_csv("training.csv")
test = pd.read_csv("test.csv")

/var/folders/7n/r8rrh9ss6q9b_lyhlmx1x_hw0000gn/T/ipykernel_80093/3416410604.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  training = pd.read_csv("training.csv")
/var/folders/7n/r8rrh9ss6q9b_lyhlmx1x_hw0000gn/T/ipykernel_80093/3416410604.py:2: DtypeWarning: Columns (28,139) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("test.csv")


In [80]:
all_data = training.append(test, ignore_index=True)

all_data['outcome'] = (all_data['metric_locked_percent_first_114_lockable_days_v1']>0.7).astype(int)
all_data = all_data.drop(columns=['finance_order_id','metric_locked_percent_first_114_lockable_days_v1'])
all_data['weekend'] = all_data['time_day_of_week_v1'].apply(lambda x: x in [5,6,7]).astype(int)
all_data.head(10)

/var/folders/7n/r8rrh9ss6q9b_lyhlmx1x_hw0000gn/T/ipykernel_80093/929078636.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = training.append(test, ignore_index=True)


,application_time,clerk_loan_count_total_v1,clerk_loan_count_last_7_days_v1,clerk_loan_count_last_28_days_v1,clerk_loan_days_since_first_v1,clerk_loan_days_since_last_v1,clerk_fpd_1_last_7_mature_days_v1,clerk_fpd_3_last_7_mature_days_v1,clerk_fpd_7_last_14_mature_days_v1,clerk_fpd_15_last_28_mature_days_v1,...,credit_account_count_payment_frequency_biweekly_mx_v1,credit_account_count_payment_frequency_bimonthly_mx_v1,credit_account_count_payment_frequency_annually_mx_v1,credit_account_count_distinct_account_type_mx_v1,credit_account_count_distinct_credit_type_mx_v1,credit_account_count_distinct_creditor_mx_v1,credit_account_count_distinct_warning_mx_v1,credit_account_count_creditor_motorcyle_finance_company_mx_v1,outcome,weekend
0,1639665815,93.0,2.0,15.0,435.0,281.0,0.166667,0.000000,0.000000,0.000000,...,0,0,0,1,1,1,0,0,0,0
1,1639666973,636.0,45.0,176.0,492.0,279.0,0.023256,0.000000,0.021277,0.000000,...,0,0,0,1,1,1,0,0,0,0
2,1639667578,386.0,19.0,54.0,568.0,279.0,0.083333,0.000000,0.000000,0.022222,...,0,0,0,2,2,2,1,0,0,0
3,1639671052,15.0,4.0,15.0,295.0,280.0,0.000000,0.000000,0.000000,NaN,...,0,0,0,2,3,2,1,0,1,0
4,1639671273,1263.0,30.0,104.0,964.0,279.0,0.151515,0.090909,0.000000,0.000000,...,0,0,0,3,3,3,2,0,0,0
5,1639671698,924.0,56.0,143.0,792.0,279.0,0.000000,0.000000,0.019231,0.010870,...,0,0,0,1,1,1,0,0,0,0
6,1639671710,579.0,15.0,65.0,801.0,280.0,0.100000,0.050000,0.000000,0.000000,...,0,0,0,1,2,2,1,0,0,0
7,1639672338,123.0,5.0,13.0,557.0,279.0,0.000000,0.000000,0.000000,0.125000,...,18,0,0,2,3,4,1,0,0,0
8,1639673380,453.0,11.0,51.0,908.0,279.0,0.250000,0.133333,0.076923,0.016949,...,0,0,0,1,1,2,0,0,1,0
9,1639673841,63.0,9.0,42.0,340.0,279.0,0.071429,0.000000,0.000000,0.045455,...,0,0,0,4,4,5,1,0,0,0


In [81]:
features=all_data.columns
#Remove 'outcome from features
features = features.drop('outcome')
print(features)

Index(['application_time', 'clerk_loan_count_total_v1',
       'clerk_loan_count_last_7_days_v1', 'clerk_loan_count_last_28_days_v1',
       'clerk_loan_days_since_first_v1', 'clerk_loan_days_since_last_v1',
       'clerk_fpd_1_last_7_mature_days_v1',
       'clerk_fpd_3_last_7_mature_days_v1',
       'clerk_fpd_7_last_14_mature_days_v1',
       'clerk_fpd_15_last_28_mature_days_v1',
       ...
       'credit_account_count_payment_frequency_monthly_mx_v1',
       'credit_account_count_payment_frequency_biweekly_mx_v1',
       'credit_account_count_payment_frequency_bimonthly_mx_v1',
       'credit_account_count_payment_frequency_annually_mx_v1',
       'credit_account_count_distinct_account_type_mx_v1',
       'credit_account_count_distinct_credit_type_mx_v1',
       'credit_account_count_distinct_creditor_mx_v1',
       'credit_account_count_distinct_warning_mx_v1',
       'credit_account_count_creditor_motorcyle_finance_company_mx_v1',
       'weekend'],
      dtype='object', length=

In [82]:
#split features into categorical and numerical
cat_features = all_data[features].select_dtypes(include=['object']).columns
num_features = all_data[features].select_dtypes(exclude=['object']).columns

print(cat_features)
print(num_features)

Index(['merchant_state_v1', 'merchant_region_mx_v1', 'user_state_v1',
       'user_region_mx_v1',
       'user_photo_similarity_score_max_is_over_99_point_993_mx_v1',
       'credit_inquiry_days_since_last_at_most_1_mx_v1'],
      dtype='object')
Index(['application_time', 'clerk_loan_count_total_v1',
       'clerk_loan_count_last_7_days_v1', 'clerk_loan_count_last_28_days_v1',
       'clerk_loan_days_since_first_v1', 'clerk_loan_days_since_last_v1',
       'clerk_fpd_1_last_7_mature_days_v1',
       'clerk_fpd_3_last_7_mature_days_v1',
       'clerk_fpd_7_last_14_mature_days_v1',
       'clerk_fpd_15_last_28_mature_days_v1',
       ...
       'credit_account_count_payment_frequency_monthly_mx_v1',
       'credit_account_count_payment_frequency_biweekly_mx_v1',
       'credit_account_count_payment_frequency_bimonthly_mx_v1',
       'credit_account_count_payment_frequency_annually_mx_v1',
       'credit_account_count_distinct_account_type_mx_v1',
       'credit_account_count_distinct_cr

In [83]:
all_data_cat = pd.get_dummies(all_data[cat_features], drop_first = True)

#drop the categorical features from the original dataset
all_data = all_data.drop(cat_features, axis = 1)

#concatenate the dummy variables with the original dataset
all_data_binned = pd.concat([all_data, all_data_cat], axis = 1)

#store only the categorical features in a list
all_data_cat.rename(columns={'merchant_state_v1_nuevo león':'merchant_state_v1_nuevoLeon'},inplace=True)
all_data_cat['merchant_state_v1_nuevoLeon'] = all_data_cat['merchant_state_v1_nuevoLeon'] + all_data_cat['merchant_state_v1_nuevo leon']
all_data_cat = all_data_cat.drop(columns=['merchant_state_v1_nuevo leon'])
cat_features = all_data_cat.columns

all_data_binned.rename(columns={'merchant_state_v1_nuevo león':'merchant_state_v1_nuevoLeon'},inplace=True)

#merge colums 'merchant_state_v1_nuevo leon' and 'merchant_state_v1_nuevoLeon'
all_data_binned['merchant_state_v1_nuevoLeon'] = all_data_binned['merchant_state_v1_nuevoLeon'] + all_data_binned['merchant_state_v1_nuevo leon']
all_data_binned = all_data_binned.drop(columns=['merchant_state_v1_nuevo leon'])

all_data_binned.head(10)


,application_time,clerk_loan_count_total_v1,clerk_loan_count_last_7_days_v1,clerk_loan_count_last_28_days_v1,clerk_loan_days_since_first_v1,clerk_loan_days_since_last_v1,clerk_fpd_1_last_7_mature_days_v1,clerk_fpd_3_last_7_mature_days_v1,clerk_fpd_7_last_14_mature_days_v1,clerk_fpd_15_last_28_mature_days_v1,...,user_region_mx_v1_region_2,user_region_mx_v1_region_3,user_region_mx_v1_region_4,user_region_mx_v1_region_5,user_region_mx_v1_region_6,user_region_mx_v1_region_7,user_region_mx_v1_region_8,user_region_mx_v1_region_9,user_photo_similarity_score_max_is_over_99_point_993_mx_v1_True,credit_inquiry_days_since_last_at_most_1_mx_v1_True
0,1639665815,93.0,2.0,15.0,435.0,281.0,0.166667,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,1,0,0,0
1,1639666973,636.0,45.0,176.0,492.0,279.0,0.023256,0.000000,0.021277,0.000000,...,0,0,0,0,0,0,1,0,0,0
2,1639667578,386.0,19.0,54.0,568.0,279.0,0.083333,0.000000,0.000000,0.022222,...,0,0,0,0,0,0,1,0,0,0
3,1639671052,15.0,4.0,15.0,295.0,280.0,0.000000,0.000000,0.000000,NaN,...,0,0,0,1,0,0,0,0,1,0
4,1639671273,1263.0,30.0,104.0,964.0,279.0,0.151515,0.090909,0.000000,0.000000,...,0,0,0,0,0,1,0,0,0,0
5,1639671698,924.0,56.0,143.0,792.0,279.0,0.000000,0.000000,0.019231,0.010870,...,0,1,0,0,0,0,0,0,0,0
6,1639671710,579.0,15.0,65.0,801.0,280.0,0.100000,0.050000,0.000000,0.000000,...,0,1,0,0,0,0,0,0,0,0
7,1639672338,123.0,5.0,13.0,557.0,279.0,0.000000,0.000000,0.000000,0.125000,...,0,0,1,0,0,0,0,0,0,0
8,1639673380,453.0,11.0,51.0,908.0,279.0,0.250000,0.133333,0.076923,0.016949,...,0,0,0,1,0,0,0,0,0,0
9,1639673841,63.0,9.0,42.0,340.0,279.0,0.071429,0.000000,0.000000,0.045455,...,0,0,1,0,0,0,0,0,0,0


In [105]:
training_processed = all_data_binned.iloc[0:len(training),:]
test_processed = all_data_binned.iloc[len(training):len(all_data_binned),:]

In [106]:
print(all_data_binned.isna().sum().sort_values(ascending=False))
print(training_processed.isna().sum().sort_values(ascending=False))

listNAN=training_processed.isna().sum().sort_values(ascending=False)

listNAN.to_csv("ListNANTraining.csv",index = False)

credit_inquiry_days_since_last_creditor_editorial_mx_v1                   50000
credit_delinquent_days_max_max_type_charge_card_last_24_months_mx_v1      50000
credit_balance_total_type_home_equity_loan_updated_last_6_months_mx_v1    50000
credit_account_opening_days_since_last_type_charge_card_mx_v1             50000
credit_account_opening_days_since_last_type_home_equity_loan_mx_v1        50000
                                                                          ...  
credit_inquiry_count_creditor_convenience_store_last_6_months_mx_v1           0
credit_inquiry_count_creditor_convenience_store_last_1_months_mx_v1           0
credit_inquiry_count_creditor_community_finance_society_mx_v1                 0
credit_inquiry_count_creditor_communications_mx_v1                            0
credit_inquiry_days_since_last_at_most_1_mx_v1_True                           0
Length: 1400, dtype: int64
credit_account_opening_days_since_last_type_home_equity_loan_mx_v1           25000
credit_bal

In [107]:
#print columns with less than 5% of missing values
print(listNAN[listNAN<0.05*len(training_processed)])

#get columns with less than 5% of missing values
listNAN=listNAN[listNAN<0.05*len(training_processed)]

#drop the rows from the training data set with less than 5% of missing values
training_processed = training_processed.dropna(subset=listNAN.index)

print(len(training_processed))

training_processed.head(10)

merchant_fpd_1_last_28_mature_days_v1                                  1217
user_zip_first_3_digits_score_mx_v1                                     966
merchant_loan_count_total_v1                                            343
merchant_loan_count_last_7_days_v1                                      343
merchant_loan_count_last_28_days_v1                                     343
                                                                       ... 
credit_inquiry_count_creditor_convenience_store_last_6_months_mx_v1       0
credit_inquiry_count_creditor_convenience_store_last_1_months_mx_v1       0
credit_inquiry_count_creditor_community_finance_society_mx_v1             0
credit_inquiry_count_creditor_communications_mx_v1                        0
credit_inquiry_days_since_last_at_most_1_mx_v1_True                       0
Length: 689, dtype: int64
22834


,application_time,clerk_loan_count_total_v1,clerk_loan_count_last_7_days_v1,clerk_loan_count_last_28_days_v1,clerk_loan_days_since_first_v1,clerk_loan_days_since_last_v1,clerk_fpd_1_last_7_mature_days_v1,clerk_fpd_3_last_7_mature_days_v1,clerk_fpd_7_last_14_mature_days_v1,clerk_fpd_15_last_28_mature_days_v1,...,user_region_mx_v1_region_2,user_region_mx_v1_region_3,user_region_mx_v1_region_4,user_region_mx_v1_region_5,user_region_mx_v1_region_6,user_region_mx_v1_region_7,user_region_mx_v1_region_8,user_region_mx_v1_region_9,user_photo_similarity_score_max_is_over_99_point_993_mx_v1_True,credit_inquiry_days_since_last_at_most_1_mx_v1_True
0,1639665815,93.0,2.0,15.0,435.0,281.0,0.166667,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,1,0,0,0
1,1639666973,636.0,45.0,176.0,492.0,279.0,0.023256,0.000000,0.021277,0.000000,...,0,0,0,0,0,0,1,0,0,0
2,1639667578,386.0,19.0,54.0,568.0,279.0,0.083333,0.000000,0.000000,0.022222,...,0,0,0,0,0,0,1,0,0,0
3,1639671052,15.0,4.0,15.0,295.0,280.0,0.000000,0.000000,0.000000,NaN,...,0,0,0,1,0,0,0,0,1,0
4,1639671273,1263.0,30.0,104.0,964.0,279.0,0.151515,0.090909,0.000000,0.000000,...,0,0,0,0,0,1,0,0,0,0
5,1639671698,924.0,56.0,143.0,792.0,279.0,0.000000,0.000000,0.019231,0.010870,...,0,1,0,0,0,0,0,0,0,0
6,1639671710,579.0,15.0,65.0,801.0,280.0,0.100000,0.050000,0.000000,0.000000,...,0,1,0,0,0,0,0,0,0,0
7,1639672338,123.0,5.0,13.0,557.0,279.0,0.000000,0.000000,0.000000,0.125000,...,0,0,1,0,0,0,0,0,0,0
8,1639673380,453.0,11.0,51.0,908.0,279.0,0.250000,0.133333,0.076923,0.016949,...,0,0,0,1,0,0,0,0,0,0
9,1639673841,63.0,9.0,42.0,340.0,279.0,0.071429,0.000000,0.000000,0.045455,...,0,0,1,0,0,0,0,0,0,0


In [108]:
listNAN=training_processed.isna().sum().sort_values(ascending=False)

listNAN.to_csv("ListNANTraining.csv",index = False)


In [109]:
features=training_processed.columns
#Remove 'outcome from features
features = features.drop('outcome')
print(features)

Index(['application_time', 'clerk_loan_count_total_v1',
       'clerk_loan_count_last_7_days_v1', 'clerk_loan_count_last_28_days_v1',
       'clerk_loan_days_since_first_v1', 'clerk_loan_days_since_last_v1',
       'clerk_fpd_1_last_7_mature_days_v1',
       'clerk_fpd_3_last_7_mature_days_v1',
       'clerk_fpd_7_last_14_mature_days_v1',
       'clerk_fpd_15_last_28_mature_days_v1',
       ...
       'user_region_mx_v1_region_2', 'user_region_mx_v1_region_3',
       'user_region_mx_v1_region_4', 'user_region_mx_v1_region_5',
       'user_region_mx_v1_region_6', 'user_region_mx_v1_region_7',
       'user_region_mx_v1_region_8', 'user_region_mx_v1_region_9',
       'user_photo_similarity_score_max_is_over_99_point_993_mx_v1_True',
       'credit_inquiry_days_since_last_at_most_1_mx_v1_True'],
      dtype='object', length=1399)


In [110]:
print(cat_features)
print(num_features)

Index(['merchant_state_v1_bc', 'merchant_state_v1_bcs',
       'merchant_state_v1_camp', 'merchant_state_v1_cdmx',
       'merchant_state_v1_chih', 'merchant_state_v1_chis',
       'merchant_state_v1_coah', 'merchant_state_v1_col',
       'merchant_state_v1_dgo', 'merchant_state_v1_gro',
       'merchant_state_v1_gto', 'merchant_state_v1_hgo',
       'merchant_state_v1_jal', 'merchant_state_v1_mex',
       'merchant_state_v1_mich', 'merchant_state_v1_mor',
       'merchant_state_v1_nay', 'merchant_state_v1_nl',
       'merchant_state_v1_nuevoLeon', 'merchant_state_v1_oax',
       'merchant_state_v1_pue', 'merchant_state_v1_qro',
       'merchant_state_v1_qroo', 'merchant_state_v1_sin',
       'merchant_state_v1_slp', 'merchant_state_v1_son',
       'merchant_state_v1_tab', 'merchant_state_v1_tamp',
       'merchant_state_v1_tlax', 'merchant_state_v1_ver',
       'merchant_state_v1_yuc', 'merchant_state_v1_zac',
       'merchant_region_mx_v1_region_2', 'merchant_region_mx_v1_region_3',


In [111]:
#separate training_processed into training_processed_cat and training_processed_num
training_processed_cat = training_processed[cat_features]
training_processed_num = training_processed[num_features]

In [114]:
#impute missing values in training_processed_num with the mean using simple imputer
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(training_processed_num)
training_processed_num = pd.DataFrame(imp.transform(training_processed_num), columns=training_processed_num.columns)

training_processed_num.head(10)



ValueError: Shape of passed values is (22834, 1259), indices imply (22834, 1317)

In [113]:
#With SimpleImputer, fill the missing values with the mean of the column in the numerical dataset and with the most frequent value in the categorical dataset


print(X_train)

[[1.63966582e+09 9.30000000e+01 2.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.63966697e+09 6.36000000e+02 4.50000000e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.63966758e+09 3.86000000e+02 1.90000000e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.65248904e+09 5.20000000e+02 1.20000000e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.65248922e+09 1.04000000e+02 7.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.65248947e+09 2.70000000e+01 5.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.